# Build datasets
- select a subset of the original data
- perform negative sampling

In [1]:
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [62]:
apps = pd.read_csv('./job-recommendation/apps.tsv', delimiter='\t',encoding='utf-8')
user_history = pd.read_csv('./job-recommendation/user_history.tsv', delimiter='\t',encoding='utf-8')
jobs = pd.read_csv('./job-recommendation/jobs.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False)
users = pd.read_csv('./job-recommendation/users.tsv' ,delimiter='\t',encoding='utf-8')
# test_users = pd.read_csv('./job-recommendation/test_users.tsv', delimiter='\t',encoding='utf-8')

b'Skipping line 122433: expected 11 fields, saw 12\n'
b'Skipping line 602576: expected 11 fields, saw 12\n'
b'Skipping line 990950: expected 11 fields, saw 12\n'


# 1. Select the subset where ```WindowID=6```

In [63]:
user_history[user_history.WindowID==6].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193853 entries, 1337041 to 1530893
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   UserID    193853 non-null  int64 
 1   WindowID  193853 non-null  int64 
 2   Split     193853 non-null  object
 3   Sequence  193853 non-null  int64 
 4   JobTitle  180658 non-null  object
dtypes: int64(3), object(2)
memory usage: 8.9+ MB


In [64]:
jobs[jobs.WindowID==6].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115998 entries, 861371 to 977368
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   JobID         115998 non-null  int64 
 1   WindowID      115998 non-null  int64 
 2   Title         115996 non-null  object
 3   Description   115997 non-null  object
 4   Requirements  115930 non-null  object
 5   City          115998 non-null  object
 6   State         115998 non-null  object
 7   Country       115998 non-null  object
 8   Zip5          71528 non-null   object
 9   StartDate     115998 non-null  object
 10  EndDate       115998 non-null  object
dtypes: int64(2), object(9)
memory usage: 10.6+ MB


In [65]:
users[users.WindowID==6].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43334 entries, 296639 to 339972
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   UserID                43334 non-null  int64  
 1   WindowID              43334 non-null  int64  
 2   Split                 43334 non-null  object 
 3   City                  43334 non-null  object 
 4   State                 43276 non-null  object 
 5   Country               43334 non-null  object 
 6   ZipCode               43142 non-null  object 
 7   DegreeType            43334 non-null  object 
 8   Major                 32433 non-null  object 
 9   GraduationDate        29703 non-null  object 
 10  WorkHistoryCount      43334 non-null  int64  
 11  TotalYearsExperience  41733 non-null  float64
 12  CurrentlyEmployed     40653 non-null  object 
 13  ManagedOthers         43334 non-null  object 
 14  ManagedHowMany        43334 non-null  int64  
dtypes: float64(1)

In [67]:
user_set = users[(users.WindowID==6) & (users.Country=="US")].dropna(axis=0,subset=["Major", "TotalYearsExperience", "CurrentlyEmployed"])
user_set.Split.value_counts(), user_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30011 entries, 296639 to 339971
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   UserID                30011 non-null  int64  
 1   WindowID              30011 non-null  int64  
 2   Split                 30011 non-null  object 
 3   City                  30011 non-null  object 
 4   State                 30011 non-null  object 
 5   Country               30011 non-null  object 
 6   ZipCode               29965 non-null  object 
 7   DegreeType            30011 non-null  object 
 8   Major                 30011 non-null  object 
 9   GraduationDate        23384 non-null  object 
 10  WorkHistoryCount      30011 non-null  int64  
 11  TotalYearsExperience  30011 non-null  float64
 12  CurrentlyEmployed     30011 non-null  object 
 13  ManagedOthers         30011 non-null  object 
 14  ManagedHowMany        30011 non-null  int64  
dtypes: float64(1)

(Train    28869
 Test      1142
 Name: Split, dtype: int64,
 None)

In [68]:
user_set.UserID.unique().size

30011

In [69]:
job_set = jobs[(jobs.WindowID==6) & (jobs.Country=="US")].dropna(axis=0,subset=["Description","Requirements","Title"])
job_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115691 entries, 861371 to 977368
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   JobID         115691 non-null  int64 
 1   WindowID      115691 non-null  int64 
 2   Title         115691 non-null  object
 3   Description   115691 non-null  object
 4   Requirements  115691 non-null  object
 5   City          115691 non-null  object
 6   State         115691 non-null  object
 7   Country       115691 non-null  object
 8   Zip5          71509 non-null   object
 9   StartDate     115691 non-null  object
 10  EndDate       115691 non-null  object
dtypes: int64(2), object(9)
memory usage: 10.6+ MB


In [70]:
user_id = user_set.UserID.unique().tolist()
job_id = job_set.JobID.unique().tolist()
work_history = user_history[user_history.UserID.isin(user_id)]
work_history.dropna(axis=0,subset=["JobTitle"], inplace=True)

application_record = apps[(apps.UserID.isin(user_id))&(apps.JobID.isin(job_id))]
work_history.info(), application_record.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 130791 entries, 1337041 to 1530886
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   UserID    130791 non-null  int64 
 1   WindowID  130791 non-null  int64 
 2   Split     130791 non-null  object
 3   Sequence  130791 non-null  int64 
 4   JobTitle  130791 non-null  object
dtypes: int64(3), object(2)
memory usage: 6.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 121442 entries, 1247132 to 1421276
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   UserID           121442 non-null  int64 
 1   WindowID         121442 non-null  int64 
 2   Split            121442 non-null  object
 3   ApplicationDate  121442 non-null  object
 4   JobID            121442 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 5.6+ MB


(None, None)

- filter out users who are not in work_history or application_record

In [73]:
work_user_id = work_history.UserID.unique()
application_user_id = application_record.UserID.unique()
user_set = user_set[(user_set.UserID.isin(work_user_id))&(user_set.UserID.isin(application_user_id))]
user_id = user_set.UserID.unique()
application_record = application_record[application_record.UserID.isin(user_id)]
work_history = work_history[work_history.UserID.isin(user_id)]

In [75]:
application_record.info(), work_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120457 entries, 1247132 to 1421276
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   UserID           120457 non-null  int64 
 1   WindowID         120457 non-null  int64 
 2   Split            120457 non-null  object
 3   ApplicationDate  120457 non-null  object
 4   JobID            120457 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 5.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 112294 entries, 1337041 to 1530886
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   UserID    112294 non-null  int64 
 1   WindowID  112294 non-null  int64 
 2   Split     112294 non-null  object
 3   Sequence  112294 non-null  int64 
 4   JobTitle  112294 non-null  object
dtypes: int64(3), object(2)
memory usage: 5.1+ MB


(None, None)

In [76]:
application_record.Split.value_counts(), user_set.Split.value_counts()

(Train    111488
 Test       8969
 Name: Split, dtype: int64,
 Train    24888
 Test       628
 Name: Split, dtype: int64)

In [79]:
(user_set.UserID.unique() == application_record.UserID.unique()).all(), (user_set.UserID.unique() == work_history.UserID.unique()).all()

(True, True)

In [80]:
user_set.to_csv("user_set.csv", index=False)
application_record.to_csv("application_record.csv", index=False)
work_history.to_csv("work_history.csv", index=False)
job_set.to_csv("job_set.csv", index=False)

# 2. Negative Sampling

In [134]:
dataset_sample = pd.DataFrame(columns = ["UserID","JobID","label"])
job_id = job_set.JobID.unique().tolist()
groups = application_record.groupby("UserID")
user_ids = []
job_ids = []
labels = []

for idx, group in tqdm(groups):
    size = len(group)
    exist_job = group.JobID.unique().tolist()
    candidate_job = [i for i in job_id if i not in exist_job ]
    sample_job = np.random.randint(0,len(candidate_job),size)
    user_ids.extend([idx] * 2 * size)
    exist_job.extend([candidate_job[i] for i in sample_job])
    job_ids.extend(exist_job)
    label = [1] * size
    label.extend([0] * size)
    labels.extend(label)

dataset_sample.UserID = user_ids
dataset_sample.JobID = job_ids
dataset_sample.label = labels

100%|██████████| 25516/25516 [04:46<00:00, 89.04it/s] 


In [138]:
dataset_sample.label.value_counts()

1    120457
0    120457
Name: label, dtype: int64

In [139]:
dataset_sample.to_csv("dataset.csv",index=False)

# 3. Filter out the labels in html

In [67]:
job_set = pd.read_csv("job_set.csv")

In [68]:
import re

def filter_tags(htmlstr):

    re_cdata=re.compile('//<!\[CDATA\[[^>]*//\]\]>',re.I) 
    re_script=re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>',re.I)#Script
    re_style=re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>',re.I)#style
    re_br=re.compile('<br\s*?/?>')
    re_h=re.compile('</?\w+[^>]*>')
    re_comment=re.compile('<!--[^>]*-->')
    s=re_cdata.sub('',htmlstr)
    s=re_script.sub('',s) 
    s=re_style.sub('',s)
    s=re_br.sub('\n',s)
    s=re_h.sub('',s) 
    s=re_comment.sub('',s)
    blank_line=re.compile('\n+')
    s=blank_line.sub('\n',s)
    s=s.replace('\\r'," ")
    s=s.replace('\\t'," ")
    s=s.replace('\n'," ")
    s=s.replace('\\n'," ")
    s = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', s, flags=re.MULTILINE)
    s = re.sub(r'[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?', '', s, flags=re.MULTILINE)
    s = re.sub(r'(www)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', s, flags=re.MULTILINE)
    s = re.sub(r'[0-9a-zA-Z.]+@[0-9a-zA-Z.]', " ", s, flags=re.MULTILINE)
    s = re.sub('\xa0', " ", s, flags=re.MULTILINE)
    s=replaceCharEntity(s)
    return s


def replaceCharEntity(htmlstr):
    CHAR_ENTITIES={'nbsp':' ','160':' ',
        'lt':'<','60':'<',
        'gt':'>','62':'>',
        'amp':'&','38':'&',
        'quot':'"','34':'"',}
   
    re_charEntity=re.compile(r'&#?(?P<name>\w+);')
    sz=re_charEntity.search(htmlstr)
    while sz:
        entity=sz.group()
        key=sz.group('name')
        try:
            htmlstr=re_charEntity.sub(CHAR_ENTITIES[key],htmlstr,1)
            sz=re_charEntity.search(htmlstr)
        except KeyError:
            htmlstr=re_charEntity.sub('',htmlstr,1)
            sz=re_charEntity.search(htmlstr)
    return htmlstr
def repalce(s,re_exp,repl_string):
    return re_exp.sub(repl_string,s)

In [70]:
Description = [filter_tags(i) for i in job_set.Description.values]
Requirements = [filter_tags(i) for i in job_set.Requirements.values]

In [71]:
job_set.drop(columns=["Description","Requirements"])
job_set["Description"] = Description
job_set["Requirements"] = Requirements

In [73]:
job_set.to_csv("job_set_cleaned.csv", index=False)